In [34]:

import pandas as pd

data = pd.read_csv('Processed__Features_Stock.csv')
data['Date'] = pd.to_datetime(data['Date'])

# start date is a month prior to Lehman Brothers filing for bankruptcy
start_date_housing = '2008-08-15'  

# end date is marking of recovery one month after the index hit its lowest point
end_date_housing = '2009-04-09'  

filtered_data_housing_crash = data[(data['Date'] >= start_date_housing) & (data['Date'] <= end_date_housing)]

# Drop rows with missing values
data_cleaned = filtered_data_housing_crash.dropna(subset=['Date', 'symbol', 'Ratio', 'Grp_Ratio', 'Vol_Diff'])

# Convert numeric columns integers
numeric_columns = ['Ratio', 'Grp_Ratio', 'Vol_Diff', 'Log_5DayMovAvg', 
                   'Roll_Rank', 'Forced_Index', 'Balanced_Vol', 
                   'Mass_Index', 'DPO', 'ichimoku_a']

data_cleaned[numeric_columns] = data_cleaned[numeric_columns].apply(pd.to_numeric, errors='coerce')

data_cleaned = data_cleaned.dropna()

data_cleaned

,Date,symbol,Ratio,Grp_Ratio,Vol_Diff,Log_5DayMovAvg,Roll_Rank,Forced_Index,Balanced_Vol,Mass_Index,DPO,ichimoku_a
188656,2008-08-15,ABC,0.001458,0.003516,-730400.0,14.838134,0.955,-49712.0,1.802021e+10,25.914987,-0.0145,41.137500
188657,2008-08-15,ACN,-0.002432,0.005379,511400.0,14.759619,0.770,-486304.0,2.205499e+09,24.124087,0.7355,40.560000
188658,2008-08-15,ADP,0.005942,0.002206,360300.0,14.460119,0.825,341297.0,4.614223e+10,25.078317,-0.6180,43.817175
188659,2008-08-15,AEE,0.013269,0.004118,63200.0,14.179362,0.265,-14222.0,1.369322e+10,25.672774,0.5225,40.699900
188660,2008-08-15,AEP,0.009963,0.000262,-177100.0,14.666678,0.410,-18101.0,1.493560e+10,25.584240,0.2785,39.072500
...,...,...,...,...,...,...,...,...,...,...,...,...
191639,2009-04-08,YUM,0.020035,0.005907,563000.0,15.673541,0.830,-134272.0,1.836312e+11,24.968497,-0.1220,28.092500
191640,2009-04-09,DUK,-0.007112,0.005004,1028200.0,14.744781,0.225,-68680.0,9.109069e+10,23.889763,0.2115,13.627300
191641,2009-04-09,ED,-0.007402,0.006163,262700.0,14.603386,0.220,-78166.0,8.145497e+10,24.797888,-0.0410,37.585000
191642,2009-04-09,KMB,0.005004,0.005029,-321500.0,14.860005,0.140,-34452.0,1.282301e+11,24.616491,-0.5750,46.920000
